<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [13]</a>'.</span>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import yfinance as yf
import datetime

from valueinvesting import utils as utils

# 1. Load raw input
## Hardcoded block

In [2]:
evaluate_last_X_years = True
X=10
currency_pair = 'EUR=X'
numbers_in_currency = 'EUR'
share_name = 'FORTUM.HE'

# 1.1 Calculate Share numbers from total distributed dividend (CFS) and dividend amount

# 1.2. Raw input statement data

In [3]:
route = f"../data/input/{share_name}_data.csv"
data = pd.read_csv(route, sep=';', parse_dates=['date', 'real_date'])
# replace -, extend missing 0-s, fix datatype
data = utils.replace_format_input(data)

In [4]:
owned_shares = pd.read_csv('../data/extras/owned_stocks.csv', sep=';', parse_dates=['date'])
owned_shares = owned_shares.loc[(owned_shares['stock'] == share_name) & (owned_shares['active'] == 'yes')]

## (OPTIONAL) Time Filtering - Last 7 years daat

In [5]:
if evaluate_last_X_years:
    data = data[data['date'] > datetime.datetime.today() - datetime.timedelta(days=X*366+93)]
    data = data.reset_index(drop=True)

data.head()

,date,revenue,cogs,gross_profit,net_profit,cash,acc_rec,inventory,curr_assets,goodwill,...,acc_pay,short_term_debt,current_debt,curr_liab,long_term_debt,total_liab,cash_from_operating_activities,capex,shares,real_date
0,2014-12-31,1133,-473,660,523,2009,830,256,4301,170,...,0,287,816,2067,5881,10511,452,-200,888.3,2015-03-01
1,2015-03-31,1040,-433,607,295,1773,637,247,7212,197,...,0,214,824,3698,5944,11761,603,-101,888.3,2015-05-10
2,2015-06-30,794,-363,431,118,2774,612,270,9857,197,...,0,176,1560,2543,5029,9659,296,-108,888.3,2015-08-09
3,2015-09-30,661,-312,349,-658,2390,574,258,9238,168,...,0,250,836,1836,5010,8793,151,-138,888.3,2015-11-09
4,2015-12-31,964,-407,557,18,3289,822,231,9610,0,...,0,204,838,2042,4965,8973,331,-180,888.3,2016-02-29


## Calculate date when quaterly reports could be available

In [6]:
data = utils.calculate_real_date(data)

# 2. Filter out unnecesary columns

In [7]:
data = utils.filter_raw_data(data)
data.tail()

,date,real_date,shares,revenue,cogs,gross_profit,net_profit,cash,acc_rec,inventory,...,intangible_assets,total_assets,acc_pay,short_term_debt,current_debt,curr_liab,long_term_debt,total_liab,cash_from_operating_activities,capex
36,2023-12-31,2024-02-29,897.2,1858,-1106,752,412,4183,1286,452,...,643,18739,1181,507,961,3708,4476,10300,149,-130
37,2024-03-31,2024-05-10,897.2,2015,-1167,848,473,4875,1030,405,...,619,18956,0,580,1815,4484,4039,10566,538,-110
38,2024-06-30,2024-08-12,897.2,1255,-696,559,215,4058,796,411,...,619,17796,0,456,556,2361,4829,9240,338,-108
39,2024-09-30,2024-10-28,897.2,1094,-611,483,133,4255,692,348,...,560,17729,0,420,509,2149,4843,8923,349,-133
40,2024-12-31,2025-02-10,897.2,1435,-821,614,338,4136,1007,420,...,549,17307,956,333,492,1877,4336,8153,167,-121


## Pull historical average USD national currency rates and add to the dataframe

In [8]:
data['usd_nat_currency'] = utils.daily_price(
    ticker=currency_pair,
    end=data['date'],
    days_earlier=90
    )

# drop rows, when USD rates wasn't available
data = data[data['usd_nat_currency'].notna()]

data.tail()

,date,real_date,shares,revenue,cogs,gross_profit,net_profit,cash,acc_rec,inventory,...,total_assets,acc_pay,short_term_debt,current_debt,curr_liab,long_term_debt,total_liab,cash_from_operating_activities,capex,usd_nat_currency
36,2023-12-31,2024-02-29,897.2,1858,-1106,752,412,4183,1286,452,...,18739,1181,507,961,3708,4476,10300,149,-130,0.929674
37,2024-03-31,2024-05-10,897.2,2015,-1167,848,473,4875,1030,405,...,18956,0,580,1815,4484,4039,10566,538,-110,0.920544
38,2024-06-30,2024-08-12,897.2,1255,-696,559,215,4058,796,411,...,17796,0,456,556,2361,4829,9240,338,-108,0.928647
39,2024-09-30,2024-10-28,897.2,1094,-611,483,133,4255,692,348,...,17729,0,420,509,2149,4843,8923,349,-133,0.910617
40,2024-12-31,2025-02-10,897.2,1435,-821,614,338,4136,1007,420,...,17307,956,333,492,1877,4336,8153,167,-121,0.937043


## Convert USD values related to IS, BS, CFS to national currency

In [9]:
# convert columns into national currency if necessary
if numbers_in_currency == 'USD':
    data_nat_curr = utils.convert_national_currency(input_data=data, currency=data)
else:
    data_nat_curr = data.copy()
data_nat_curr.tail()

,date,real_date,shares,revenue,cogs,gross_profit,net_profit,cash,acc_rec,inventory,...,total_assets,acc_pay,short_term_debt,current_debt,curr_liab,long_term_debt,total_liab,cash_from_operating_activities,capex,usd_nat_currency
36,2023-12-31,2024-02-29,897.2,1858,-1106,752,412,4183,1286,452,...,18739,1181,507,961,3708,4476,10300,149,-130,0.929674
37,2024-03-31,2024-05-10,897.2,2015,-1167,848,473,4875,1030,405,...,18956,0,580,1815,4484,4039,10566,538,-110,0.920544
38,2024-06-30,2024-08-12,897.2,1255,-696,559,215,4058,796,411,...,17796,0,456,556,2361,4829,9240,338,-108,0.928647
39,2024-09-30,2024-10-28,897.2,1094,-611,483,133,4255,692,348,...,17729,0,420,509,2149,4843,8923,349,-133,0.910617
40,2024-12-31,2025-02-10,897.2,1435,-821,614,338,4136,1007,420,...,17307,956,333,492,1877,4336,8153,167,-121,0.937043


# 3. Calculate input to Value Raios (eps, bv per share, fcf)
## filter out unwanted columns

In [10]:
filtered_nat_curr = utils.calculate_input_value_ratios(data_nat_curr)

## Calculate input values to value ratios

In [11]:
ratios_nat_curr = utils.ratios_input_filter(filtered_nat_curr)

## Calculate performance parameters (current ratio, ROA, ROE, margins, D/E)

In [12]:
ratios_nat_curr = utils.evaluate_performance(input=filtered_nat_curr, output=ratios_nat_curr)

# 4. Pull historical share prices and merge with sattement data

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [13]:
# pull weekly share prices and merge with the value ratios
merged_nat_curr = utils.add_share_prices_to_value_ratios(share_name, data, ratios_nat_curr)
merged_nat_curr.head()

$FORTUM.HE: possibly delisted; no price data found  (1wk 2014-12-31 00:00:00 -> 2025-03-22 23:36:30.545010)


AttributeError: Can only use .dt accessor with datetimelike values

# Calculate price ratios (P/E, P/B, P/S, P/FCF)

In [ ]:
merged_nat_curr = utils.price_ratios(merged_nat_curr)
merged_nat_curr.head()

# Calculate Dividend yield and currency-share price correlation

In [ ]:
# get daily historical USD-national currency rates
usd_nat_curr = utils.get_historical_currency_rate(currency_pair, merged_nat_curr)

In [ ]:
# get share prices and dividend yield data
ticker_all_price = utils.get_historical_share_dividend(share_name, merged_nat_curr)

In [ ]:
# Pearson's correlation coefficient between share price and USD - national currency rate
utils.get_currency_share_price_correlation(share_name, usd_nat_curr, ticker_all_price)

# Share price and value ratios

In [ ]:
for column in merged_nat_curr.columns[3:]:
    fig, ax1 = plt.subplots(figsize=(15, 6))

    ax2 = ax1.twinx()
    ax1.plot(merged_nat_curr['real_date'], merged_nat_curr[column], color='k', label=merged_nat_curr)
    ax2.plot(ticker_all_price.index, ticker_all_price['Close'], color='b', label=share_name)

    ax1.set_xlabel('Date')
    ax1.set_ylabel(column, color='k')
    ax2.set_ylabel('Share price (national currency)', color='b')

    plt.legend()
    plt.show()

In [ ]:
# plot histograms of value parameters
utils.plot_histogram_value_parameters(input_df=merged_nat_curr, extra_parameters=[], owned_shares=owned_shares)

## Calculate correlation between value parameters

In [ ]:
# calulate value parameters Pearson's correlation coefficient
pd.DataFrame(merged_nat_curr, columns=['roa', 'roe', 'pe_ratio', 'pb_ratio', 'ps_ratio', 'ev_revenue', 'debt_to_equity', 'current_ratio']).corr()

In [ ]:
final_df = utils.get_value_stock_target_prices(
    input_df=merged_nat_curr,
    min_pct=0.2,
    max_pct=0.5,
    owned_shares=owned_shares
)